In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
# Data
from ucimlrepo import fetch_ucirepo
# Quantum
from qiskit.circuit.library import ZZFeatureMap
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.algorithms import PegasosQSVC
from qiskit_algorithms.optimizers import SPSA
from qiskit.primitives import Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit import QuantumCircuit
from qiskit.circuit.library import BlueprintCircuit

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [57]:
wine = fetch_ucirepo(id=109)

In [69]:
X = pd.DataFrame(wine.data.targets)
y = pd.DataFrame(wine.data.features)
print(type(X))

<class 'pandas.core.frame.DataFrame'>


In [70]:
X_array = X.to_numpy()
n_qubits = X_array.shape[1]
qc = QuantumCircuit(n_qubits)
for row in X_array:
    for i, feature in enumerate(row):
        qc.ry(feature * np.pi, i)

print(qc)

   ┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐»
q: ┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├»
   └───────┘└───────┘└───────┘└───────┘└───────┘└───────┘└───────┘└───────┘»
«   ┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐»
«q: ┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├»
«   └───────┘└───────┘└───────┘└───────┘└───────┘└───────┘└───────┘└───────┘»
«   ┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐»
«q: ┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├»
«   └───────┘└───────┘└───────┘└───────┘└───────┘└───────┘└───────┘└───────┘»
«   ┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐»
«q: ┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├┤ Ry(π) ├»
«   └───────┘└───────┘└───────┘└───────┘└───────┘└───────┘└───────┘└───────┘»
«   ┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐

# Pre-Processing of data

In [75]:
# Separating features for both datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

((142, 1), (36, 1))

In [79]:
scaler = StandardScaler()
# Scale the features
# Using the same scaler to avoid data leakage
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Custom Feature Map
class CustomFeatureMap(BlueprintCircuit):
    def __init__(self, feature_dimension):
        super().__init__()
        self.feature_dimension = feature_dimension
        self.num_qubits = feature_dimension
        self.parameters = [f"theta_{i}" for i in range(feature_dimension)]
        self._num_parameters = feature_dimension
        self.build()

    def build(self):
        qc = QuantumCircuit(self.feature_dimension)
        for i in range(self.feature_dimension):
            qc.ry(self.parameters[i], i)
        self.compose(qc, inplace=True)

custom_feature_map = CustomFeatureMap(feature_dimension = X_train.shape[1])

sampler = Sampler()

# Subroutine that leverages the Sampler primitive
fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=custom_feature_map) 

qsvc = QSVC(quantum_kernel=kernel, max_iter=100)

quantum_instance = AerSimulator(method='statevector', shots = 1024)

print("Training QSVM...")
qsvc.fit(X_train, y_train)

qsvc_score = qsvc.score(X_test, y_test)
print("QSVC Score:", qsvc_score)

y_pred = qsvc.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

TypeError: Can't instantiate abstract class CustomFeatureMap without an implementation for abstract methods '_build', '_check_configuration'

# Quantum Support Vector Machine

In [24]:
feature_map = ZZFeatureMap(
    feature_dimension = X_train.shape[1],
    reps = 2,
    entanglement = 'linear'
)

sampler = Sampler()

# Subroutine that leverages the Sampler primitive
fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map) 

# Selecting a simulating method
quantum_instance = AerSimulator(method='statevector', shots = 1024)

qsvc = QSVC(quantum_kernel=kernel, max_iter=100)

print("Training QSVM...")
qsvc.fit(X_train, y_train)

qsvc_score = qsvc.score(X_test, y_test)
print("QSVC Score:", qsvc_score)

y_pred = qsvc.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/var/folders/pf/wyb5vhpd027g120tcw057sb80000gn/T/ipykernel_68661/1023482254.py:7: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/pf/wyb5vhpd027g120tcw057sb80000gn/T/ipykernel_68661/1023482254.py:10: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  fidelity = ComputeUncompute(sampler=sampler)
/Users/fibu/anaconda3/envs/jupyterlab/lib/python3.12/site-packages/sklearn/svm/_base.py:737: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

Training QSVM...


/Users/fibu/anaconda3/envs/jupyterlab/lib/python3.12/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


QSVC Score: 0.4444444444444444

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        14
           2       0.44      1.00      0.62        16
           3       0.00      0.00      0.00         6

    accuracy                           0.44        36
   macro avg       0.15      0.33      0.21        36
weighted avg       0.20      0.44      0.27        36



/Users/fibu/anaconda3/envs/jupyterlab/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/fibu/anaconda3/envs/jupyterlab/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/fibu/anaconda3/envs/jupyterlab/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

# Pegasos Quantum Support Vector Machine

In [38]:
y_train.shape

(142, 1)

In [39]:
# Number of qubits is equal to the number of features
num_qubits = 2

# Number of steps to perform during training
tau = 100

# Regularization parameter
C = 1000

feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1)

qkernel = FidelityQuantumKernel(feature_map=feature_map)

pegasos_qsvc = PegasosQSVC(quantum_kernel = qkernel, C=C, num_steps=tau)

# Train the model
pegasos_qsvc.fit(X_train, y_train)

pegasos_score = pegasos_qsvc.score(X_test, y_test)

print(f"PegasosQSVC classification test score {pegasos_score}")

ValueError: y has to be a 1D array